# Introduction to Data Science – Homework 6
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

Due: Friday, February 23, 11:59pm.

In Part 1 of this homework you will scrape github repositories and organize the information in a Pandas dataframe. In Part 2, you will use linear regression to gain meaningful insights. 

## Your Data
First Name:
<br>
Last Name:
<br>
E-mail:
<br>
UID:
<br>

In [1]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import requests
import urllib.request


import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

### 1. Scrape Github Repository List and Repository Information using BeautifulSoup
In this part you will explore over 2 million Github repositories. You are going to scrape [this repository list](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories).

### 1.1. Check whether you are permitted to scrape the data
Before you start to scrape any website you should go through the terms and services or similar pages of the website. Almost all websites post conditions to use their data. Check the terms of [https://github.com/](https://github.com/) to see whether the site permits you to scrape their data or not. Are you sure you are allowed to scrape?

**Your Interpretation:**
TODO

### Task 1.2 Download the website

Download the **first ten** pages of the list of highly starred repositories here: 

https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories

To avoid sending too many requests to the server, make sure that your querying code is in a sperate cell from anything else, so that you don't have to re-run it needlessly. 

**Warning:** while we haven't experienced any problems, exepct that github will temporarily block access if you scrape their website relentlessly. Make sure to be responsible. Also, don't do this last minute – if you're blocked for e.g., 24 hours that could impact your timeline.

Now read the html text in python and create a soup object.

In [2]:
## Your code goes here

# download the fist ten pages


url = "https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=stars&type=Repositories"

with urllib.request.urlopen(url) as response:
    html = response.read()
    html = html.decode('utf-8')

#A simple solution is to just use ‘response.text’ to get html returned by requests.get(url) directly. This doesn't work on Windows.

# html = requests.get(url)
# html = html.text

# # save the file
with open('git_repositories.html', 'w') as new_file:
    new_file.write(str(html.replace('\n','').encode('utf-8')))

In [3]:
# Creating the soup
git_soup = BeautifulSoup(html, 'html.parser')
git_soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://assets-cdn.github.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-7a12427f1445.css" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-fca0cb55603c.css" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/site-e1e1bc98a53e.css" media="all" rel="stylesheet">
<meta content="width=device-width" name="vi

### 1.3. Extract Data

Extract the foolowing data for each repository, and create a Pandas Dataframe with a row for each repository and a column for each of these datums. 

1. Name of the repository
+ URL of the repository 
+ Number of Contributors 
+ Programming Language Used 
+ Number of stars 
+ Number of issues
+ Number of files 
+ Length of readme file.

Note that you will frist have to extract links from the soup that you just scraped earlier, and then download the repository pages to retreive most of the data. 

Refer to lecture 12 for details on how to do this. Make sure to use the web inspector to identify the relevant structures. 

Save the dataframe you created to a new file `project_info.csv` and include this in your submission.

In [75]:
## Your code goes here

name_list = [] #
url_list = [] #
cont_list = [] #
lang_list = [] # 
stars_list = []
issues_list = []
forks_list = []
rmlength_list = []

repos = git_soup.find_all("div",class_="col-8 pr-3")
repos_lang = git_soup.find_all("div",class_="d-table-cell col-2 text-gray pt-2")
url = "https://github.com"

for i in range(10):
    name_list.append(repos[i].find("a",class_="v-align-middle").text)
    url_tail = repos[i].find("a").get("href")
    repo_url = url + url_tail
    url_list.append(repo_url)
    with urllib.request.urlopen(repo_url) as response:
        repo_html = response.read()
        repo_html = repo_html.decode('utf-8')
    # if i%5==0:
    #     time.sleep(5) #sleeps after every 5 request
    #     # adjust frequency or time for sleeping if have issues
    with open('repo.html', 'w') as new_file:
        new_file.write(str(repo_html.replace('\n','').encode('utf-8')))
    repo_soup = BeautifulSoup(repo_html, 'html.parser')
    cont_list.append(repo_soup.find("a", href= lambda x : x and "contributors" in x).text.replace('\n','').strip('contributors').strip())
    lang_list.append(repos_lang[i].text.strip())
    stars_list.append(repo_soup.find("a",class_="social-count js-social-count").text.strip())
    issues_list.append(repo_soup.find("span",class_="Counter").text)
    forks_list.append(repo_soup.find("a", href= lambda x : x and "network" in x).text.strip())
    rmlength_list.append(len(repo_soup.find("article",class_="markdown-body entry-content").text.replace("\n","")))
    
print('names:',name_list,'\n')
print('URLs:',url_list,'\n')
print('cont_list:',cont_list,'\n')
print('lang_list:',lang_list,'\n')
print('# of stars:',stars_list,'\n')
print('# of issues:',issues_list,'\n')
print('# of forks:',forks_list,'\n')
print('RM lengths:',rmlength_list,'\n')


names: ['freeCodeCamp/freeCodeCamp'] 

URLs: ['https://github.com/freeCodeCamp/freeCodeCamp'] 

cont_list: ['731'] 

lang_list: ['JavaScript'] 

# of stars: ['291,624'] 

# of issues: ['165'] 

# of forks: ['13,480'] 

RM lengths: [2739] 



In [76]:
repo_soup.find("article",class_="markdown-body entry-content").text.replace("\n","")


"Welcome to freeCodeCamp's open source codebase and curriculum!freeCodeCamp is a friendly open source community where you learn to code and help nonprofits.We help our campers build job-worthy portfolios of real apps used by real people, while helping nonprofits.You start by working through our self-paced, browser-based full stack JavaScript curriculum.By working through our curriculum, you can earn four certifications:1. Front End CertificationThe first section will teach you the basics of how webpages work and also introduce you to JavaScript programming.Skills you'll practice include HTML, CSS, JavaScript, jQuery and Bootstrap.To earn this certification, you'll build 10 front-end projects and implement many JavaScript algorithms.2. Data Visualization CertificationThe second section builds upon the first and introduces you to more advanced topics such as Sass, React and D3.To earn this certification, you'll build 5 React-apps and 5 Data visualization apps using D3.js.3. Back End Cert

In [49]:
# repos[0].find("div",class_="d-table-cell col-2 text-gray pt-2")
repos[0]
forks_list

['\n    8,402\n  ']

In [5]:
# Convert list into DataFrame

In [6]:
# Save dataframe to file project_info.csv

### 2. Use linear regression to analyze the Github repository data

In this part, you will analyze the data collectd in Part 1 using regression tools. The goal is to identify properties that make a repository popular. 

First, load the project_info.csv file in again. **We need you to do this so that we can run your code below withouth having to run your scraping code.**

In [7]:
# this loads the data from the project_info.csv file 
project_info = pd.read_csv('project_info.csv')
project_info.head()

FileNotFoundError: File b'project_info.csv' does not exist

### 2.1. Reformat the data into useful datatypes

1. Print the data types for the columns of your dataframe
+ Reformat the colums Issues, Forks, Stars, and Contributers to be integer data
+ There is one repository flagged as having infinite contributers (the Linux kernel). We'll assume that it in fact has 15000 contributors (about twice as much as the next project). 

In [ ]:
# your code goes here


### Task 2.2 Describe the data

+ Get an overview of the data using the describe function.
+ Compute the correlation matrix, visualize it with a heat map.
+ Visualize the correlations by making a scatterplot matrix.
+ Interprete what you see.

You can re-use code from your previous homework here.

In [ ]:
# your code goes here

**Your Interpretation:** TODO

### 2.3. Linear regression

1. First use linear regression to try to predict the number of Stars based on Forks, Contributors, Issues, and Readme Length. Explain why this is not a very good model by discussing the R-squared , F-statistic p-value, and coefficient  p-values. 
+ Develop another model which is better. Explain why it is better and interpret your results. 

In [ ]:
# your code goes here

** Your interpretation:** TODO